In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models, layers, optimizers

# 1. Ladda datasetet (MNIST finns inbyggt i Keras)

In [3]:
# Detta ger oss träningsdata (för att lära modellen) och testdata (för att utvärdera den)
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# 2. Utforska datans dimensioner

In [4]:
# X_train form: (60000, 28, 28) -> 60 000 bilder, 28x28 pixlar
print(f"Träningsdata form: {X_train.shape}")
print(f"Antal klasser: {len(np.unique(y_train))}") # Borde vara 10 (siffrorna 0-9)

Träningsdata form: (60000, 28, 28)
Antal klasser: 10


# 3. Normalisering (Viktigt!)

In [5]:
# Neurala nätverk gillar små tal (ofta mellan 0 och 1).
# Vi delar med 255.0 eftersom pixelvärdena är 0-255.
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# 4. Flattening (Platta till bilderna)

In [7]:
# Ett standard ANN (Dense layer) kan inte se 2D-mönster (som ett rutnät).
# Vi måste göra om 28x28 matrisen till en lång rad (vektor) med 784 siffror (28*28=784).
X_train_flat = X_train.reshape((-1, 784))
X_test_flat = X_test.reshape((-1, 784))

print(f"Form efter flattening: {X_train_flat.shape}")

Form efter flattening: (60000, 784)


# 5. Bygga ANN-modellen

In [10]:
# Skapa modellen
model = Sequential([
    # Input layer: Anger att vi tar emot 784 värden
    Input(shape=(784,)),

    # Hidden Layer 1: 128 neuroner, ReLU aktivering
    Dense(128, activation='relu'),

    # Hidden Layer 2: 64 neuroner, ReLU aktivering (valfritt, men kan hjälpa)
    Dense(64, activation='relu'),

    # Output Layer: 10 neuroner (0-9), Softmax för sannolikheter
    Dense(10, activation='softmax')
])

# Kompilera modellen
# Optimizer 'adam': En smartare variant av Gradient Descent som justerar inlärningstakten själv.
# Loss 'sparse_categorical_crossentropy': Används när vi har flera klasser (0-9) som heltal (inte one-hot encoded).
# Metrics 'accuracy': Vi vill se hur många procent rätt vi har.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Sammanfattning av modellen
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

# 6. Träning

In [11]:
# Träna modellen
# epochs=5: Vi går igenom hela datasetet 5 gånger.
# validation_split=0.2: 20% av träningsdatan används för att 'fusktesta' under träningens gång.
history = model.fit(X_train_flat, y_train, epochs=5, validation_split=0.2)

# Utvärdera på testdatan (data modellen aldrig sett förut)
test_loss, test_acc = model.evaluate(X_test_flat, y_test)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9215 - loss: 0.2665 - val_accuracy: 0.9551 - val_loss: 0.1528
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9652 - loss: 0.1164 - val_accuracy: 0.9660 - val_loss: 0.1173
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9754 - loss: 0.0784 - val_accuracy: 0.9725 - val_loss: 0.0942
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9813 - loss: 0.0586 - val_accuracy: 0.9728 - val_loss: 0.0907
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9854 - loss: 0.0473 - val_accuracy: 0.9736 - val_loss: 0.0948
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9723 - loss: 0.0929
Test accuracy: 0.9723


# 7. Optimering med Keras Tuner

In [15]:
# 1. Definiera en funktion som bygger modellen
# 'hp' står för HyperParameters - ett objekt vi får från tunern
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))

    # --- HYPERPARAMETER 1: Antal neuroner ---
    # Vi säger till tunern: "Välj ett heltal (Int) mellan 32 och 512, i steg om 32"
    # hp.Int skapar ett spann som tunern kan testa ifrån.
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

    model.add(layers.Dense(units=hp_units, activation='relu'))

    # (Vi kan lägga till fler lager här om vi vill, men vi håller det enkelt nu)

    model.add(layers.Dense(10, activation='softmax'))

    # --- HYPERPARAMETER 2: Learning Rate ---
    # Vi säger: "Testa dessa specifika värden för learning rate"
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# 7. Starta Tunern

In [16]:
# Hyperband är en smart algoritm som snabbt slänger dåliga modeller och satsar resurser på de lovande.
tuner = kt.Hyperband(
    build_model,             # Funktionen vi skrev ovan
    objective='val_accuracy', # Vi vill maximera träffsäkerheten på valideringsdatan
    max_epochs=10,           # Max antal epoker per test
    factor=3,
    directory='my_dir',      # Mapp där resultaten sparas
    project_name='intro_to_kt'
)

# 3. Lägg till Early Stopping (Nu använder vi det!)
# Om modellen inte blir bättre på 3 epoker, avbryt just det testet.
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 4. Kör sökningen! (Detta kan ta en stund)
print("Startar sökning")
tuner.search(X_train_flat, y_train,
             epochs=10,
             validation_split=0.2,
             callbacks=[stop_early])

# 5. Hämta den bästa modellen och dess hyperparametrar
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Sökningen är klar! 
Det optimala antalet neuroner i första lagret var {best_hps.get('units')} 
och den bästa learning rate var {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 44s]
val_accuracy: 0.9782500267028809

Best val_accuracy So Far: 0.9804999828338623
Total elapsed time: 00h 10m 10s

Sökningen är klar! 
Det optimala antalet neuroner i första lagret var 512 
och den bästa learning rate var 0.001.



# 8. Bygga modellen

In [21]:


# --- BYGG DEN VINNANDE MODELLEN MANUELLT ---
# (Baserat på resultatet från Keras Tuner: 512 units, lr=0.001)

final_model = models.Sequential([
    # Input-lager (platta datan)
    layers.Input(shape=(784,)),

    # Det vinnande dolda lagret (512 neuroner)
    layers.Dense(512, activation='relu'),

    # Output-lager (alltid 10 för MNIST)
    layers.Dense(10, activation='softmax')
])

# Kompilera med den vinnande learning rate (0.001)
# Vi skapar optimeraren separat för att kunna sätta learning_rate manuellt
opt = optimizers.Adam(learning_rate=0.001)

final_model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

final_model.summary()

# --- TRÄNA MODELLEN ---
# Nu tränar vi den "skarpt" med lite mer tålamod
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = final_model.fit(X_train_flat, y_train,
                          epochs=50,
                          validation_split=0.2,
                          callbacks=[early_stop])

print("Modellen är klar och tränad!")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9355 - loss: 0.2218 - val_accuracy: 0.9612 - val_loss: 0.1301
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9746 - loss: 0.0856 - val_accuracy: 0.9712 - val_loss: 0.0945
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9822 - loss: 0.0573 - val_accuracy: 0.9718 - val_loss: 0.0955
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9871 - loss: 0.0404 - val_accuracy: 0.9790 - val_loss: 0.0722
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9905 - loss: 0.0286 - val_accuracy: 0.9781 - val_loss: 0.0747
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9929 - loss: 0.0216 - val_accuracy: 0.9764 - val_loss: 0.0939
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9945 - loss: 0.0179 - val_accuracy: 0.9793 - val_loss: 0.0813
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9956 - loss: 0.0136 - 